In [5]:
import tensorflow as tf
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [6]:
df = pd.read_csv('customer_churn_dataset-testing-master.csv')
df.head()

,CustomerID,Age,Gender,Tenure,Usage Frequency,Support Calls,Payment Delay,Subscription Type,Contract Length,Total Spend,Last Interaction,Churn
0,1,22,Female,25,14,4,27,Basic,Monthly,598,9,1
1,2,41,Female,28,28,7,13,Standard,Monthly,584,20,0
2,3,47,Male,27,10,2,29,Premium,Annual,757,21,0
3,4,35,Male,9,12,5,17,Premium,Quarterly,232,18,0
4,5,53,Female,58,24,9,2,Standard,Annual,533,18,0


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64374 entries, 0 to 64373
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   CustomerID         64374 non-null  int64 
 1   Age                64374 non-null  int64 
 2   Gender             64374 non-null  object
 3   Tenure             64374 non-null  int64 
 4   Usage Frequency    64374 non-null  int64 
 5   Support Calls      64374 non-null  int64 
 6   Payment Delay      64374 non-null  int64 
 7   Subscription Type  64374 non-null  object
 8   Contract Length    64374 non-null  object
 9   Total Spend        64374 non-null  int64 
 10  Last Interaction   64374 non-null  int64 
 11  Churn              64374 non-null  int64 
dtypes: int64(9), object(3)
memory usage: 5.9+ MB


In [9]:
df.isnull().sum()

CustomerID           0
Age                  0
Gender               0
Tenure               0
Usage Frequency      0
Support Calls        0
Payment Delay        0
Subscription Type    0
Contract Length      0
Total Spend          0
Last Interaction     0
Churn                0
dtype: int64

In [10]:
df.duplicated().sum()

0

In [11]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['Gender'] = le.fit_transform(df['Gender'])
df['Subscription Type'] = le.fit_transform(df['Subscription Type'])
df['Contract Length'] = le.fit_transform(df['Contract Length'])

In [12]:
df.drop('CustomerID',axis=1,inplace=True)

In [8]:
df['Churn'].value_counts()

0    33881
1    30493
Name: Churn, dtype: int64

In [13]:
x = df.drop('Churn',axis=1)
y = df['Churn']

In [14]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(x,y,random_state=1,test_size=0.3)

In [16]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
xtrain = ss.fit_transform(xtrain)
xtest = ss.transform(xtest)

In [17]:
xtrain.shape

(45061, 10)

In [18]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=11,activation='relu',input_shape=(10,)),
    tf.keras.layers.Dense(units=1,activation='sigmoid')
])

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 11)                121       
                                                                 
 dense_1 (Dense)             (None, 1)                 12        
                                                                 
Total params: 133
Trainable params: 133
Non-trainable params: 0
_________________________________________________________________


In [21]:
model.compile(optimizer='sgd',loss='binary_crossentropy',metrics=['accuracy'])

In [22]:
from tensorflow.keras.callbacks import EarlyStopping

cb = EarlyStopping(
    monitor = 'val_loss',   # testing error
    min_delta = 0.00001,    # min value of delta
    patience = 20,
    verbose = 1,            # how many iterations
    mode = 'auto',          # stop at min loss
    baseline = None,
    restore_best_weights = False
)

In [23]:
trained_model = model.fit(xtrain,ytrain,epochs=5000,callbacks=cb,validation_data=(xtest,ytest))

Epoch 1/5000
1409/1409 [==============================] - 5s 3ms/step - loss: 0.4793 - accuracy: 0.7711 - val_loss: 0.3902 - val_accuracy: 0.8237
Epoch 2/5000
1409/1409 [==============================] - 3s 2ms/step - loss: 0.3736 - accuracy: 0.8316 - val_loss: 0.3632 - val_accuracy: 0.8355
Epoch 3/5000
1409/1409 [==============================] - 3s 2ms/step - loss: 0.3535 - accuracy: 0.8413 - val_loss: 0.3465 - val_accuracy: 0.8440
Epoch 4/5000
1409/1409 [==============================] - 4s 3ms/step - loss: 0.3386 - accuracy: 0.8500 - val_loss: 0.3329 - val_accuracy: 0.8539
Epoch 5/5000
1409/1409 [==============================] - 3s 2ms/step - loss: 0.3259 - accuracy: 0.8570 - val_loss: 0.3209 - val_accuracy: 0.8598
Epoch 6/5000
1409/1409 [==============================] - 4s 3ms/step - loss: 0.3141 - accuracy: 0.8638 - val_loss: 0.3102 - val_accuracy: 0.8642
Epoch 7/5000
1409/1409 [==============================] - 3s 2ms/step - loss: 0.3035 - accuracy: 0.8675 - val_loss: 0.3000 -

In [24]:
print('Training_loss :',model.evaluate(xtrain,ytrain))
print('Testing_loss :',model.evaluate(xtest,ytest))

1409/1409 [==============================] - 2s 1ms/step - loss: 0.1283 - accuracy: 0.9505
Training_loss : [0.12831543385982513, 0.9504671692848206]
604/604 [==============================] - 1s 1ms/step - loss: 0.1364 - accuracy: 0.9443
Testing_loss : [0.13638153672218323, 0.9443380236625671]


In [27]:
Y_pred = model.predict(xtest).round(2)

604/604 [==============================] - 1s 1ms/step


In [28]:
Y_pred

array([[0.02],
       [0.98],
       [1.  ],
       ...,
       [0.6 ],
       [0.03],
       [0.  ]], dtype=float32)

In [29]:
xtest

array([[-0.71726854, -0.93481121, -1.40191125, ...,  0.01047368,
        -1.13037265, -1.2174844 ],
       [ 0.57314186,  1.06973471,  0.35064635, ...,  0.01047368,
         1.16804837,  0.86620324],
       [ 0.64483133, -0.93481121, -0.11670234, ...,  0.01047368,
        -1.0116209 , -1.33324482],
       ...,
       [-1.36247374,  1.06973471,  1.22692515, ...,  0.01047368,
         1.51664223, -1.33324482],
       [ 0.71652079,  1.06973471,  0.05855342, ..., -1.22500892,
         0.40573873, -1.68052609],
       [-1.50585267,  1.06973471, -0.17512093, ..., -1.22500892,
        -1.57856475,  0.05588027]])

In [30]:
Y_pred = np.where(Y_pred>=0.5,1,0)
Y_pred

array([[0],
       [1],
       [1],
       ...,
       [1],
       [0],
       [0]])

In [32]:
from sklearn.metrics import classification_report,confusion_matrix

print(classification_report(ytest,Y_pred))
print(confusion_matrix(ytest,Y_pred))

              precision    recall  f1-score   support

           0       0.96      0.94      0.95     10171
           1       0.93      0.95      0.94      9142

    accuracy                           0.94     19313
   macro avg       0.94      0.94      0.94     19313
weighted avg       0.94      0.94      0.94     19313

[[9531  640]
 [ 437 8705]]
